### Reader

In [ ]:
import os
import xml.etree.ElementTree as ET
cache_path = '/home/zhengzheng/scratch0/projects/Fine-Tuned-GPT-2-with-articles-ground-truth/code/llamaIndex/.cache'
filenames = [filename for filename in os.listdir(cache_path) if filename.endswith('tei.xml')]


def get_full_text(element):
    text = element.text or ''
    for subelement in element:
        text += ET.tostring(subelement, encoding='unicode', method='text')
        if subelement.tail:
            text += subelement.tail
    return text.strip()

iter_filenames = (iter(filenames))

In [ ]:
file_path = os.path.join(cache_path, next(iter_filenames)) 
file_path

In [ ]:
file_dict = {}
tree = ET.parse(file_path)
root = tree.getroot()

namespace = {'tei': 'http://www.tei-c.org/ns/1.0'}

In [ ]:
# title
title = root.find('.//tei:teiHeader/tei:fileDesc/tei:titleStmt/tei:title', namespaces=namespace)
file_dict['title'] = title.text
file_dict

In [ ]:
# authors
authors = root.findall('.//tei:teiHeader/tei:fileDesc/tei:sourceDesc/tei:biblStruct/tei:analytic/tei:author/tei:persName', namespaces=namespace)
file_dict['authors'] = []
for author in authors:
    forename = author.findall('tei:forename', namespaces=namespace)
    if len(forename) == 0:
        continue
    forename = ' '.join(name.text for name in forename)
    surname = ' '.join([name.text for name in author.findall('tei:surname', namespaces=namespace)])
    author_name = forename + ' ' + surname
    file_dict['authors'].append(author_name)
file_dict

In [ ]:
# abstract
abstract = root.find('.//tei:teiHeader/tei:profileDesc/tei:abstract/tei:div/tei:p', namespaces=namespace)
print(abstract)
if abstract is not None:
    file_dict['abstract'] = abstract.text
file_dict

In [ ]:
# Body
body = root.findall('.//tei:text/tei:body/tei:div', namespaces=namespace)

for i, child in enumerate(body):
    ps = child.findall('.//tei:p', namespaces=namespace)
    if len(ps) == 0:
        continue
    head = child.find('.//tei:head', namespaces=namespace).text
    content = '\n'.join([get_full_text(p) for p in ps])
    file_dict[head] = content
file_dict

In [ ]:
from utils.custom_document_reader import CustomDocumentReader

documents = CustomDocumentReader(
    input_dir="/home/zhengzheng/scratch0/projects/Fine-Tuned-GPT-2-with-articles-ground-truth/data",
    cache_dir="/home/zhengzheng/scratch0/projects/Fine-Tuned-GPT-2-with-articles-ground-truth/code/llamaIndex/.cache",
    config_path="/home/zhengzheng/scratch0/projects/Fine-Tuned-GPT-2-with-articles-ground-truth/code/llamaIndex/utils/config.json",
    remove_cache=False
)._load_data()

### CustomSplitter

In [ ]:
from llama_index.core.node_parser import SentenceSplitter

ss1 = SentenceSplitter(
    chunk_size=1024,
    include_metadata=True,
    include_prev_next_rel=True,
)

In [ ]:
from llama_index.core.storage.docstore import SimpleDocumentStore
nodes = ss1.get_nodes_from_documents(documents)
docstore = SimpleDocumentStore()
docstore.add_documents(nodes)
docstore.persist(persist_path="/home/zhengzheng/scratch0/projects/Fine-Tuned-GPT-2-with-articles-ground-truth/code/llamaIndex/.cache/test.json")


In [ ]:
from llama_index.core.node_parser import SimpleFileNodeParser

nodes = SimpleFileNodeParser().get_nodes_from_documents(documents=documents)

In [ ]:
for k, v in nodes[0].to_dict().items():
    print(f"key: {k}\nvalue: {v}\n\n")

In [ ]:
from utils.custom_document_reader import CustomDocumentReader

documents = CustomDocumentReader(
    input_dir="/home/zhengzheng/scratch0/projects/Fine-Tuned-GPT-2-with-articles-ground-truth/data",
    cache_dir="/home/zhengzheng/scratch0/projects/Fine-Tuned-GPT-2-with-articles-ground-truth/code/llamaIndex/.cache",
    config_path="/home/zhengzheng/scratch0/projects/Fine-Tuned-GPT-2-with-articles-ground-truth/code/llamaIndex/utils/config.json",
    remove_cache=False
)._load_data()

from utils.custom_parser import CustomHierarchicalNodeParser
hnp = CustomHierarchicalNodeParser.from_defaults()
hnp._parse_nodes(documents)

### Test custom document reader

In [ ]:
from utils.custom_document_reader import CustomDocumentReader

documents = CustomDocumentReader(
    input_dir="/home/zhengzheng/scratch0/projects/Fine-Tuned-GPT-2-with-articles-ground-truth/data",
    cache_dir="/home/zhengzheng/scratch0/projects/Fine-Tuned-GPT-2-with-articles-ground-truth/code/llamaIndex/.cache",
    config_path="/home/zhengzheng/scratch0/projects/Fine-Tuned-GPT-2-with-articles-ground-truth/code/llamaIndex/utils/config.json",
    remove_cache=False
)._load_data()

from utils.custom_parser import CustomHierarchicalNodeParser
hnp = CustomHierarchicalNodeParser.from_defaults()
nodes = hnp.get_nodes_from_documents(documents)

In [ ]:
len(documents)

In [ ]:
len(nodes)

In [ ]:
from utils.custom_extractor import HuggingfaceBasedExtractor, OllamaBasedExtractor, OpenAIBasedExtractor

extractor = OllamaBasedExtractor(
    model_name='sammcj/sfr-embedding-mistral:Q4_K_M',
    only_meta={'level': ['section']}
)

extractor.extract(nodes)

### Pydantic

In [1]:
from utils.get import get_a_store
file_path = 'D:\\Projects(D)\\Fine-Tuned-GPT-2-with-articles-ground-truth\\code\\llamaIndex\\.cache\\test-gpt-4o-batch_1_parser_CustomHierarchicalNodeParser-vicuna_13b.json'
docstore = get_a_store('SimpleDocumentStore').from_persist_path(persist_path=file_path)
nodes = [node for _, node in docstore.docs.items()][:4]

c:\Users\Zheng\miniconda3\envs\llm\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Zheng\miniconda3\envs\llm\Lib\site-packages\pydantic\_internal\_fields.py:160: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [2]:
from custom.custom_extractor import OpenAIBasedQARExtractor
from custom.schema import QAR
from dotenv import load_dotenv

load_dotenv()
extractor = OpenAIBasedQARExtractor(
    model_name='gpt-4o',
    cache_dir='D:\\Projects(D)\\Fine-Tuned-GPT-2-with-articles-ground-truth\\code\\llamaIndex\\.cache',
    mode='immediately',
    embedding_only=True,
    only_meta=None
)

In [4]:
outputs = extractor.extract(nodes, 'test', 'test_action', 'D:\\Projects(D)\\Fine-Tuned-GPT-2-with-articles-ground-truth\\code\\llamaIndex\\.cache')

100%|██████████| 4/4 [00:46<00:00, 11.60s/it]


In [ ]:
nodes[0].metadata['questions_this_excerpt_can_answer_and_corresponding_answers_and_reasons'].split('\n')

In [ ]:
nodes[0].metadata['questions_this_excerpt_can_answer_and_corresponding_answers_and_reasons'].split('\n')

In [ ]:
import re
def extract_json_str(text: str) -> list:
    """Extract JSON strings from text."""
    matches = re.findall(r'\{.*?\}', text.strip(), re.MULTILINE | re.IGNORECASE | re.DOTALL)
    if not matches:
        raise ValueError(f"Could not extract json strings from output: {text}")
    return matches

text = """```json
[
    {
        "Question": "What are the main pathological features of Parkinson's disease as described in the document?",
        "Answer": "The main pathological feature of Parkinson's disease is the progressive destruction of dopamine-producing cells in the substantia nigra region of the brain stem.",
        "Reason": "This specific detail about the pathological feature is highlighted in the context and is not general knowledge."
    },
    {
        "Question": "How does the document describe the role of neuromelanin in Parkinson's disease?",
        "Answer": "The document suggests that neuromelanin may serve as an indication of catecholamine synthesis in neuronal systems containing dopamine or norepinephrine and may influence neuronal metabolism due to the heavy metals it contains.",
        "Reason": "This insight into neuromelanin's role and its potential impact on neuronal metabolism is unique to the context provided."
    },
    {
        "Question": "What evidence does the document provide to support the hypothesis that oxidative stress is involved in Parkinson's disease?",
        "Answer": "The document provides evidence such as increased membrane peroxidation in the substantia nigra, elevated TBA-reactive substance levels, increased 8-hydroxy-2′deoxyguanosine levels, and low phospholipid turnover in the substantia nigra.",
        "Reason": "These specific pieces of evidence are detailed in the context and are not commonly known facts."
    },
    {
        "Question": "What are the potential sources of increased oxidative stress in Parkinson's disease according to the document?",
        "Answer": "Potential sources include dopamine metabolism, mitochondrial dysfunction, increased free iron levels, accumulation of AGEs, reduced activity of free radical defense systems, and the deleterious role of glial cells.",
        "Reason": "The document lists these specific sources, providing a comprehensive view that is unique to the context."
    },
    {
        "Question": "How does the document describe the involvement of mitochondrial dysfunction in Parkinson's disease?",
        "Answer": "The document describes mitochondrial dysfunction as involving abnormalities of the respiratory chain, particularly a 30-40% decrease in complex I activity in the substantia nigra, which leads to reductions in ATP production and elevated ROS generation.",
        "Reason": "This detailed description of mitochondrial dysfunction and its specific impact on Parkinson's disease is unique to the context provided."
    }
]
"""

for k in extract_json_str(text):
    print(k)
# print(extract_json_str(text))

### Read nodes

In [14]:
from llama_index.core.storage.docstore import SimpleDocumentStore
docstore = SimpleDocumentStore()
docstore.persist(persist_path="/home/zhengzheng/scratch0/projects/Fine-Tuned-GPT-2-with-articles-ground-truth/code/llamaIndex/.cache/gpt-4o-batch-all-p_2_parser_CustomHierarchicalNodeParser-hf_vicuna_13b.json")

In [15]:
nodes = [node for _, node in docstore.docs.items()]

In [16]:
len(nodes)

0